Using kernel `conda_pytorch_latest_p36`

In [1]:
# !pip install transformers
# !pip install datasets

In [2]:
import sys
sys.path.append('../../../')

In [8]:
from pathlib import Path
import os
import random
import pickle

In [84]:
import pandas as pd
import numpy as np
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from sklearn.metrics import classification_report, precision_recall_fscore_support, precision_score, f1_score, recall_score
from sklearn.preprocessing import MultiLabelBinarizer
import torch

In [5]:
from deep.constants import *

In [38]:
train = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_train.csv')
val = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_val.csv')
test = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_test.csv')

def process(df):
    df = df.copy()
    df['dimension_ids'] = df['dimension_ids'].apply(eval)
#     df['dimension_ids'] = df['dimension_ids'].apply(lambda x: torch.tensor(x, dtype=torch.float))
    
    mlb = MultiLabelBinarizer()
    labels = mlb.fit_transform(list(df['dimension_ids']))
    df['labels'] = list(labels)
    
    df = df[['excerpt', 'labels']]
    df = df.rename(columns={'excerpt': 'texts'})
        
    return df

train_df = process(train)
val_df = process(val)
test_df = process(test)

In [39]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [40]:
a = pd.read_pickle('/Users/stefano/Downloads/test_predictions.pickle')
b = sigmoid(a)

In [109]:
thresh = 0.52
index=1


preds = (b > thresh).long()
labels = torch.tensor(test_df.labels)

preds = preds[:,index]
labels = labels[:, index]

prec = precision_score(labels, preds)
rec = recall_score(labels, preds)
precision_recall_fscore_support(labels, preds, average="binary")

(0.20540074691180696, 0.5980761187787537, 0.30578424035068963, None)